In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

2022-01-11 00:10:11.419190: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.


In [2]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [3]:
tf.config.list_physical_devices('GPU')

2022-01-11 00:10:12.508707: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-11 00:10:12.509229: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-11 00:10:12.544950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 00:10:12.545291: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-11 00:10:12.545304: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-11 00:10:12.546879: I tensorflow/stream_executor/platform/defau

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

GPUs:  1


In [5]:
train = pd.read_csv('train_metadata.csv')
test = pd.read_csv('test_metadata.csv')
#submission = pd.read_csv('sample_submission.csv')

train_dir = 'train/'
test_dir = 'test/'

In [6]:
label = []
train_data = []
for i in range(train.shape[0]):
    train_data.append(train_dir + train['image_name'].iloc[i] + '.jpg')
    label.append(train['diagnosis'].iloc[i])
    
test_data = []
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image'].iloc[i] + '.jpg')

In [7]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
label_df = pd.DataFrame(label, columns = ['diagnosis'])
one_hot_encoder.fit(label_df)

label_df_encoded = one_hot_encoder.transform(label_df)
label_df_encoded = pd.DataFrame(data=label_df_encoded, columns=one_hot_encoder.categories_)
print(label_df_encoded.columns)

MultiIndex([('atypical melanocytic proliferation',),
            (               'cafe-au-lait macule',),
            (                       'lentigo NOS',),
            (               'lichenoid keratosis',),
            (                          'melanoma',),
            (                             'nevus',),
            (              'seborrheic keratosis',),
            (                     'solar lentigo',),
            (                           'unknown',)],
           )


In [8]:
df_train = pd.DataFrame(train_data)
df_train.columns = ['images']
df_train = pd.concat([df_train, label_df_encoded], axis = 1)
print(df_train.columns)
df_test = pd.DataFrame(test_data)
df_test.columns = ['images']

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_train.iloc[:, 0], df_train.iloc[:,1::], test_size = 0.3, random_state = 42)

train = pd.DataFrame(X_train)
train.columns = ['images']
train = pd.concat([train, y_train], axis = 1)

test = pd.DataFrame(X_test)
test.columns = ['images']
test = pd.concat([test, y_test], axis = 1)

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,
                                   zoom_range = 0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(300, 300),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(300, 300),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

Found 23188 validated image filenames.
Found 9938 validated image filenames.


In [11]:
model = Sequential([
        EfficientNetB3(
            input_shape=(300,300,3),
            weights='imagenet',
            include_top=False
        ),
        GlobalAveragePooling2D(),
        Dense(1024, activation = 'relu'), 
        Dropout(0.3), 
        Dense(512, activation= 'relu'), 
        Dropout(0.2), 
        Dense(256, activation='relu'), 
        Dropout(0.2), 
        Dense(128, activation='relu'), 
        Dropout(0.1), 
        Dense(9, activation='softmax')
    ])

2022-01-11 00:10:13.127837: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-11 00:10:13.128251: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-11 00:10:13.128373: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 00:10:13.128758: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-11 00:10

In [12]:
opt = Adam(lr=0.0001)

model.compile(optimizer= opt,loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb3 (Functional)  (None, 10, 10, 1536)      10783535  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1573888   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [13]:
batch_size = 128
epochs = 100
nb_train_steps = train.shape[0] // batch_size
nb_val_steps = test.shape[0] // batch_size

checkpoint = ModelCheckpoint("efficientB3_1.h5", monitor='val_accuracy', verbose=5, save_best_only=True, save_weights_only=False, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=50, verbose=1, mode='auto')

model.fit_generator(train_generator, steps_per_epoch=nb_train_steps, epochs=epochs,
                              validation_data=test_generator, validation_steps=nb_val_steps, 
                              verbose=1, callbacks=[checkpoint,early])

/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-01-11 00:10:16.178527: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-11 00:10:16.198209: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 4200000000 Hz


Epoch 1/100


2022-01-11 00:10:23.717994: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-11 00:10:23.876486: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-11 00:10:24.028974: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


181/181 [==============================] - 107s 540ms/step - loss: 0.9199 - accuracy: 0.7724 - val_loss: 0.8007 - val_accuracy: 0.8068

Epoch 00001: val_accuracy improved from -inf to 0.80682, saving model to efficientB3_1.h5
Epoch 2/100
181/181 [==============================] - 95s 522ms/step - loss: 0.3831 - accuracy: 0.9199 - val_loss: 0.7866 - val_accuracy: 0.8052

Epoch 00002: val_accuracy did not improve from 0.80682
Epoch 3/100
181/181 [==============================] - 95s 525ms/step - loss: 0.3991 - accuracy: 0.9200 - val_loss: 0.6706 - val_accuracy: 0.8068

Epoch 00003: val_accuracy did not improve from 0.80682
Epoch 4/100
181/181 [==============================] - 95s 523ms/step - loss: 0.2448 - accuracy: 0.9414 - val_loss: 0.7275 - val_accuracy: 0.8003

Epoch 00004: val_accuracy did not improve from 0.80682
Epoch 5/100
181/181 [==============================] - 92s 505ms/step - loss: 0.2947 - accuracy: 0.9285 - val_loss: 561.5774 - val_accuracy: 0.7906

Epoch 00005: val_ac

181/181 [==============================] - 91s 504ms/step - loss: 0.2572 - accuracy: 0.9453 - val_loss: 0.8034 - val_accuracy: 0.8019

Epoch 00041: val_accuracy did not improve from 0.92532
Epoch 42/100
181/181 [==============================] - 89s 491ms/step - loss: 0.2334 - accuracy: 0.9478 - val_loss: 0.5995 - val_accuracy: 0.8474

Epoch 00042: val_accuracy did not improve from 0.92532
Epoch 43/100
181/181 [==============================] - 88s 484ms/step - loss: 0.1540 - accuracy: 0.9562 - val_loss: 0.4712 - val_accuracy: 0.8523

Epoch 00043: val_accuracy did not improve from 0.92532
Epoch 44/100
181/181 [==============================] - 87s 478ms/step - loss: 0.2076 - accuracy: 0.9516 - val_loss: 0.7560 - val_accuracy: 0.8068

Epoch 00044: val_accuracy did not improve from 0.92532
Epoch 45/100
181/181 [==============================] - 86s 476ms/step - loss: 0.1812 - accuracy: 0.9516 - val_loss: 0.6038 - val_accuracy: 0.8198

Epoch 00045: val_accuracy did not improve from 0.9253

181/181 [==============================] - 85s 467ms/step - loss: 0.1703 - accuracy: 0.9553 - val_loss: 1.0103 - val_accuracy: 0.8068

Epoch 00081: val_accuracy did not improve from 0.92532
Epoch 82/100
181/181 [==============================] - 86s 472ms/step - loss: 0.2334 - accuracy: 0.9411 - val_loss: 0.2952 - val_accuracy: 0.9042

Epoch 00082: val_accuracy did not improve from 0.92532
Epoch 83/100
181/181 [==============================] - 83s 461ms/step - loss: 0.1373 - accuracy: 0.9653 - val_loss: 0.9225 - val_accuracy: 0.7224

Epoch 00083: val_accuracy did not improve from 0.92532
Epoch 84/100
181/181 [==============================] - 85s 469ms/step - loss: 0.2039 - accuracy: 0.9467 - val_loss: 0.7284 - val_accuracy: 0.8068

Epoch 00084: val_accuracy did not improve from 0.92532
Epoch 00084: early stopping
